# Lecture 03 - Student Notebook

We recommend using Noto for this lecture tutorial, where we've already installed the dependencies of the pymer4 package and statsmodels.

We extended the data with extra features. The feature description is found [here](https://docs.google.com/spreadsheets/d/15UvkrJgTapWispb6tSjMTZh0yJooOsxQ3sWIhKjYM7I/edit?usp=sharing). 

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Import the linear regression model class
#from pymer4.models import Lm

# Import the lmm model class
#from pymer4.models import Lmer

# Data directory
DATA_DIR = "./../../data"

In [4]:
# Parse the aggregated and time series data
df = pd.read_csv('{}/aggregated_extended_fc.csv'.format(DATA_DIR))
df = df.fillna('NaN')
list(df.columns)
display(df)

df_byweek = pd.read_csv('{}/fc_long_extended.csv'.format(DATA_DIR))
display(df_byweek)

,user,ch_num_sessions,ch_time_in_prob_sum,ch_time_in_video_sum,ch_ratio_clicks_weekend_day,ch_total_clicks_weekend,ch_total_clicks_weekday,ch_time_sessions_mean,ch_time_sessions_std,bo_delay_lecture,...,la_weekly_prop_watched_mean,la_weekly_prop_interrupted_mean,la_weekly_prop_interrupted_std,la_weekly_prop_replayed_mean,la_weekly_prop_replayed_std,la_frequency_action_video_play,grade,gender,category,year
0,0,1.9,2334.4,2951.8,0.850000,16.8,38.1,1392.858333,790.762032,55068.387500,...,0.245714,0.024286,0.0,0.010000,0.0,0.179203,4.50,NaN,NaN,Y2-2018-19
1,1,3.4,1698.4,9227.8,0.567500,4.0,179.4,3068.720238,1257.504407,-2883.367738,...,0.748868,0.074683,0.0,0.066456,0.0,0.332424,4.50,M,Suisse.Autres,Y2-2018-19
2,2,5.3,2340.6,10801.3,26.562274,94.6,129.2,1750.289268,1024.134043,10027.216667,...,0.354487,0.026667,0.0,0.059915,0.0,0.284407,5.25,M,Suisse.PAM,Y2-2018-19
3,3,2.8,2737.1,8185.5,3.691250,13.5,46.4,20203.590260,656.052901,27596.864484,...,0.370000,0.014286,0.0,0.020000,0.0,0.108774,4.50,F,Suisse.Autres,Y2-2018-19
4,4,2.5,3787.3,7040.0,1.543889,58.4,64.9,3373.908333,1363.320365,-914.633333,...,0.030000,0.000000,0.0,0.020000,0.0,0.199775,4.75,F,France,Y2-2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,293,3.5,8127.5,113.4,0.632304,28.9,20.6,7963.627500,1001.514794,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.034080,5.25,M,France,Y3-2019-20
284,294,2.2,2452.4,4623.1,18.147762,36.4,71.3,3614.055952,853.195566,16834.900000,...,0.140530,0.011111,0.0,0.000000,0.0,0.186649,5.25,F,France,Y3-2019-20
285,296,0.9,1643.2,1932.4,0.000000,0.4,31.2,926.916667,616.918475,-12860.522222,...,0.069231,0.023077,0.0,0.000000,0.0,0.028596,6.00,F,France,Y3-2019-20
286,297,1.4,2718.6,360.3,0.180000,2.0,15.3,346.437500,122.017326,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.032353,5.00,M,Suisse.PAM,Y3-2019-20


,Unnamed: 0,week,user,ch_num_sessions,ch_time_in_prob_sum,ch_time_in_video_sum,ch_ratio_clicks_weekend_day,ch_total_clicks_weekend,ch_total_clicks_weekday,ch_time_sessions_mean,...,la_seek_len_std,la_pause_dur_std,la_time_speeding_up_mean,la_time_speeding_up_std,la_weekly_prop_watched_mean,la_weekly_prop_interrupted_mean,la_weekly_prop_interrupted_std,la_weekly_prop_replayed_mean,la_weekly_prop_replayed_std,la_frequency_action_video_play
0,10,0,1,7.0,326.0,15525.0,5.675000,40.0,227.0,1931.285714,...,146.564097,188.175709,65.173554,150.807752,0.600000,0.100000,0.0,0.000000,0.0,0.400749
1,11,1,1,4.0,350.0,8411.0,0.000000,0.0,207.0,2190.250000,...,8.486253,78.639644,47.872928,67.365584,0.800000,0.000000,0.0,0.100000,0.0,0.391304
2,12,2,1,5.0,4577.0,8691.0,0.000000,0.0,167.0,2106.200000,...,63.484419,105.108022,64.533835,81.772612,1.000000,0.000000,0.0,0.000000,0.0,0.359281
3,13,3,1,4.0,259.0,12055.0,0.000000,0.0,239.0,3078.500000,...,31.535282,75.997314,58.085308,86.465139,0.769231,0.000000,0.0,0.153846,0.0,0.359833
4,14,4,1,3.0,480.0,13235.0,0.000000,0.0,197.0,4116.666667,...,10.594150,202.504038,78.057143,140.802708,1.000000,0.285714,0.0,0.285714,0.0,0.390863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2835,5,293,2.0,9315.0,0.0,0.513514,37.0,19.0,4657.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2336,2836,6,293,3.0,86.0,549.0,4.333333,3.0,13.0,211.666667,...,0.000000,116.639044,13.000000,9.000000,0.000000,0.000000,0.0,0.000000,0.0,0.312500
2337,2837,7,293,3.0,3675.0,0.0,0.000000,0.0,41.0,1225.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2338,2838,8,293,5.0,10956.0,0.0,0.000000,0.0,53.0,601.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000


### Predicting student performance early on

In this task, we are interested in predicting course grade early on during the semester. This type of information can be useful for an instructor in order to be able to provide intervention to struggling students. We will use again the category as a random effect.
We will need to train a separate model for each week (i.e. predicting after 1 week of the course, after 2 weeks of the course, after 3 weeks, etc.). However, we will use the same equation for all models.

First, we create a dataframe containing information about the user.

In [5]:
# parse the necessary data frames
df_ui = (df.loc[:,['user','grade','gender','category','year']]).copy()

# compute pass/fail label
df_ui['passed'] = df_ui.loc[:,'grade'] >= 4
df_ui.loc[:,'passed'] = df_ui.loc[:,'passed'].astype(int)
display(df_ui)

/tmp/ipykernel_607/2991808490.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1
 0 1 1 0 0 1 1 0 0 1 0 1 1 0 0 0 1 0 0 1 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 0
 1 0 0 0 0 1 1 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0 0
 0 0 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 0 1 1
 0 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 0 1 1 1 1 0 1 0 1
 1 0 0 1 0 0 1 1 1 1 0 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1
 1 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_ui.loc[:,'passed'] = df_ui.loc[:,'passed'].astype(int)


,user,grade,gender,category,year,passed
0,0,4.50,NaN,NaN,Y2-2018-19,1
1,1,4.50,M,Suisse.Autres,Y2-2018-19,1
2,2,5.25,M,Suisse.PAM,Y2-2018-19,1
3,3,4.50,F,Suisse.Autres,Y2-2018-19,1
4,4,4.75,F,France,Y2-2018-19,1
...,...,...,...,...,...,...
283,293,5.25,M,France,Y3-2019-20,1
284,294,5.25,F,France,Y3-2019-20,1
285,296,6.00,F,France,Y3-2019-20,1
286,297,5.00,M,Suisse.PAM,Y3-2019-20,1


Next, we reformat the data frame to contain values by week and user.

In [6]:
df_byuser = df_byweek.sort_values(by=['user', 'week']).reset_index(drop=True)
display(df_byuser)

,Unnamed: 0,week,user,ch_num_sessions,ch_time_in_prob_sum,ch_time_in_video_sum,ch_ratio_clicks_weekend_day,ch_total_clicks_weekend,ch_total_clicks_weekday,ch_time_sessions_mean,...,la_seek_len_std,la_pause_dur_std,la_time_speeding_up_mean,la_time_speeding_up_std,la_weekly_prop_watched_mean,la_weekly_prop_interrupted_mean,la_weekly_prop_interrupted_std,la_weekly_prop_replayed_mean,la_weekly_prop_replayed_std,la_frequency_action_video_play
0,10,0,1,7.0,326.0,15525.0,5.675000,40.0,227.0,1931.285714,...,146.564097,188.175709,65.173554,150.807752,0.600000,0.100000,0.0,0.000000,0.0,0.400749
1,11,1,1,4.0,350.0,8411.0,0.000000,0.0,207.0,2190.250000,...,8.486253,78.639644,47.872928,67.365584,0.800000,0.000000,0.0,0.100000,0.0,0.391304
2,12,2,1,5.0,4577.0,8691.0,0.000000,0.0,167.0,2106.200000,...,63.484419,105.108022,64.533835,81.772612,1.000000,0.000000,0.0,0.000000,0.0,0.359281
3,13,3,1,4.0,259.0,12055.0,0.000000,0.0,239.0,3078.500000,...,31.535282,75.997314,58.085308,86.465139,0.769231,0.000000,0.0,0.153846,0.0,0.359833
4,14,4,1,3.0,480.0,13235.0,0.000000,0.0,197.0,4116.666667,...,10.594150,202.504038,78.057143,140.802708,1.000000,0.285714,0.0,0.285714,0.0,0.390863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2835,5,293,2.0,9315.0,0.0,0.513514,37.0,19.0,4657.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2336,2836,6,293,3.0,86.0,549.0,4.333333,3.0,13.0,211.666667,...,0.000000,116.639044,13.000000,9.000000,0.000000,0.000000,0.0,0.000000,0.0,0.312500
2337,2837,7,293,3.0,3675.0,0.0,0.000000,0.0,41.0,1225.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2338,2838,8,293,5.0,10956.0,0.0,0.000000,0.0,53.0,601.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000


We can now create a model that predicts the exam grade after a specific number of weeks of the course. We will use 5 weeks and 10 weeks.

**Step 1**: We will write a function that aggregates the features for all weeks.

In [7]:
def aggregate_features(df_ui, df_byuser, week_nr):

    df_weeknr = df_byuser[df_byuser['week'] < week_nr]
    df_return = df_weeknr.groupby(['user']).mean()
    df_return['user'] = df_return.index
    
    # Return df with aggregated features
    df_return = df_return.set_index('user').join(df_ui.set_index('user'))
    df_return.reset_index()
    
    return df_return

**Step 2**: We will split the data into a training and test set (20% users in the test set, stratified by pass/fail label). In our case, **data stratification** refers to choosing a sample with the same ratio of pass/fail as the initial dataset, so our training set and our test set are both representative of our original population. If you are interested, you can read more about [stratifying test sets here](https://www.baeldung.com/cs/ml-stratified-sampling).

In [8]:
# perform train/test split
df_week5 = aggregate_features(df_ui, df_byuser, 5)
df_train5, df_test5 = train_test_split(df_week5, test_size=0.2, random_state=0,  stratify=df_week5['passed'])

df_week10 = aggregate_features(df_ui, df_byuser, 10)
df_train10, df_test10 = train_test_split(df_week10, test_size=0.2, random_state=0,  stratify=df_week10['passed'])

In [9]:
df_train5

,Unnamed: 0,week,ch_num_sessions,ch_time_in_prob_sum,ch_time_in_video_sum,ch_ratio_clicks_weekend_day,ch_total_clicks_weekend,ch_total_clicks_weekday,ch_time_sessions_mean,ch_time_sessions_std,...,la_weekly_prop_interrupted_mean,la_weekly_prop_interrupted_std,la_weekly_prop_replayed_mean,la_weekly_prop_replayed_std,la_frequency_action_video_play,grade,gender,category,year,passed
user,,,,,,,,,,,,,,,,,,,,,
133,1262.0,2.0,3.8,2871.2,10406.4,0.807201,144.4,57.4,3113.390000,1757.638327,...,0.028571,0.0,0.020000,0.0,0.318210,5.00,NaN,NaN,Y2-2018-19,1
5,52.0,2.0,2.2,3037.0,4369.4,0.019355,62.4,28.4,2846.550000,1906.951145,...,0.033333,0.0,0.000000,0.0,0.299758,4.00,M,Suisse.PAM,Y2-2018-19,1
97,922.0,2.0,4.2,1614.8,15358.2,1.218493,96.0,221.0,3652.283333,1765.804243,...,0.043956,0.0,0.411502,0.0,0.367533,3.50,M,France,Y2-2018-19,0
242,2332.0,2.0,4.6,4489.6,4838.2,0.327586,11.6,137.8,2013.366667,1278.815809,...,0.064935,0.0,0.057143,0.0,0.143251,4.50,F,Suisse.Autres,Y3-2019-20,1
223,2162.0,2.0,8.0,4014.8,15215.0,3.408812,32.4,176.8,1720.657302,1223.593951,...,0.505694,0.0,0.420957,0.0,0.207422,2.00,M,Suisse.PAM,Y3-2019-20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,202.0,2.0,7.0,5288.8,13880.2,1.722764,113.8,212.0,1843.900491,1666.376816,...,0.227399,0.0,0.068571,0.0,0.376160,4.50,M,France,Y2-2018-19,1
77,742.0,2.0,4.8,4484.6,13147.8,1.676138,227.8,264.8,3524.133333,2162.273626,...,0.134725,0.0,0.088571,0.0,0.402052,4.50,NaN,NaN,Y2-2018-19,1
80,762.0,2.0,1.8,868.0,5977.4,0.200000,3.0,47.2,1979.566667,1799.122890,...,0.028571,0.0,0.224908,0.0,0.158695,3.25,NaN,NaN,Y2-2018-19,0


**Step 3**: We will now train our model on the training data for 5 and 10 weeks. We will use the following formula:
```grade ~ (1|category) + wa_num_subs_perc_correct```

In [16]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Week 5 model
# Random intercept for 'category', fixed effect for 'wa_num_subs_perc_correct'
model5 = smf.mixedlm("grade ~ wa_num_subs_perc_correct", data=df_train5, groups=df_train5["category"])
result5 = model5.fit(method='nm', maxiter=500, reml=False)
print(result5.summary())

# Week 10 model
model10 = smf.mixedlm("grade ~ wa_num_subs_perc_correct", data=df_train10, groups=df_train10["category"])
result10 = model5.fit(method='nm', maxiter=500, reml=False)
print(result10.summary())

              Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    grade    
No. Observations:      187        Method:                ML       
No. Groups:            5          Scale:                 1.3404   
Min. group size:       4          Log-Likelihood:        -294.7658
Max. group size:       72         Converged:             Yes      
Mean group size:       37.4                                       
------------------------------------------------------------------
                         Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept                3.818    0.196 19.430 0.000  3.433  4.203
wa_num_subs_perc_correct 0.547    0.363  1.508 0.132 -0.164  1.259
Group Var                0.050    0.047                           

              Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    grade    
No. Observations:      1

**Step 4**: We predict on the test data and check the accuracy.

In [17]:
# predict on the test data for weeks 5, 10

from sklearn.metrics import root_mean_squared_error

predictions5 = result5.predict(df_test5)
rmse5 = root_mean_squared_error(df_test5['grade'], predictions5)

predictions10 = result10.predict(df_test10)
rmse10 = root_mean_squared_error(df_test10['grade'], predictions10)

print(rmse5)
print(rmse10)

1.1649435627340945
1.1668760482545137


## Your Turn
We are interested in predicting pass/fail (denoted by ```passed``` in the dataframe) instead of the grade.
1. Adjust the equations of ```model5``` and ```model10``` to predict pass/fail instead of the grade.
2. Train the two models, evalute their accuracy on the test data set, and send us the RMSE.

In [18]:
import requests

exec(requests.get("https://courdier.pythonanywhere.com/get-send-code").content)

npt_config = {
    'session_name': 'lecture-03',
    'session_owner': 'mlbd',
    'sender_name': input("Your name: "),
}

Your name:  Parsa


In [ ]:
# Define the model equations

# YOUR CODE HERE

In [21]:
# Fit the models
result5 = model5.fit(method='nm', maxiter=500, reml=False)
result10 = model10.fit(method='nm', maxiter=500, reml=False)
print(result5.summary())
print(result10.summary())

# predict on the test data for weeks 5, 10
predictions5 = result5.predict(df_test5)
rmse5 = str(root_mean_squared_error(df_test5['passed'], predictions5))

predictions10 = result10.predict(df_test10)
rmse10 = str(root_mean_squared_error(df_test10['passed'], predictions10))

print(rmse5)
print(rmse10)

# share the RMSEs with us
send(rmse5, 1)
send(rmse10, 2)

              Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    grade    
No. Observations:      187        Method:                ML       
No. Groups:            5          Scale:                 1.3404   
Min. group size:       4          Log-Likelihood:        -294.7658
Max. group size:       72         Converged:             Yes      
Mean group size:       37.4                                       
------------------------------------------------------------------
                         Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept                3.818    0.196 19.430 0.000  3.433  4.203
wa_num_subs_perc_correct 0.547    0.363  1.508 0.132 -0.164  1.259
Group Var                0.050    0.047                           

              Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    grade    
No. Observations:      1

<Response [200]>

Extension: if you still have time: can you improve the accuracy of the model by adding more features? Send us an explanation of why you have chosen the specific features along with the RMSE of your model.

In [22]:
# Explain briefly: what features are you adding and why?

exp = """This is an example discussion"""

### Share it with us
send(exp, 3) 

<Response [200]>

In [ ]:
# Define the model equations

# YOUR CODE HERE

In [23]:
# Fit the models
result5 = model5.fit(method='nm', maxiter=500, reml=False)
result10 = model10.fit(method='nm', maxiter=500, reml=False)
print(result5.summary())
print(result10.summary())

# predict on the test data for weeks 5, 10
predictions5 = result5.predict(df_test5)
rmse5 = str(root_mean_squared_error(df_test5['passed'], predictions5))

predictions10 = result10.predict(df_test10)
rmse10 = str(root_mean_squared_error(df_test10['passed'], predictions10))

print(rmse5)
print(rmse10)

# share the RMSEs with us
send(rmse5, 4)
send(rmse10, 5)

              Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    grade    
No. Observations:      187        Method:                ML       
No. Groups:            5          Scale:                 1.3404   
Min. group size:       4          Log-Likelihood:        -294.7658
Max. group size:       72         Converged:             Yes      
Mean group size:       37.4                                       
------------------------------------------------------------------
                         Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept                3.818    0.196 19.430 0.000  3.433  4.203
wa_num_subs_perc_correct 0.547    0.363  1.508 0.132 -0.164  1.259
Group Var                0.050    0.047                           

              Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    grade    
No. Observations:      1

<Response [200]>